In [1]:
#import packages
#pip install textblob
#pip install keras
#pip install tensorflow
import time
from textblob import TextBlob
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import numpy as np

#for CNN 
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2023-04-13 13:32:33.550578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df_10= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2010_text_wo_names.pickle")
df_10

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
3,"[say, delight, restored, bridge, back, use]",1,0,[Southease],None,"Chairman of Southease Parish, Neville Harrison...",1,2010,0
11,"[family, year, old, kill, house, fire, pay, tr...",0,2,[],None,The family of a 34-year-old mother from Bristo...,21,2010,1
13,"[family, say, kind, totally, dedicated]",1,4,[],None,Her family said she was kind and a totally ded...,21,2010,1
15,"[truly, tragic, love, family, everything, give...",0,4,[Sara],None,"'Truly tragic'""Sara loved her family above eve...",21,2010,1
16,"[everybody, know, love, miss, always]",0,3,[],None,"""Everybody who knew her will love her and miss...",21,2010,1
...,...,...,...,...,...,...,...,...,...
1919546,"[contrast, news, conference, sound, guard, eve...",2,0,[Fabio],None,Compare and contrast - Fabio Capello's news co...,2175804,2010,0
1919549,"[goal, one, chosen, five, could, take, spot, k...",2,0,"[Nelson, Mandela]",True,And after his goal at the Nelson Mandela Bay S...,2175804,2010,0
1919550,"[everyone, practise, say]",1,0,[],None,"""Everyone has been practising them,"" he said.",2175804,2010,0
1919553,"[look, relax, hair, back, cornrows, mobile, tu...",4,0,[],None,"He looked relaxed, his hair back in cornrows, ...",2175804,2010,0


In [4]:
#df_09= pd.read_pickle(r"C:\Users\danie\Documents\GitHub\Masters-Thesis\2009_preprocessed_date.pickle") 
df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2009_text_wo_names.pickle")

In [5]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

#PP Data
df_09['col_type'] = df_09.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)
df_09= df_09[df_09["col_type"].notnull()]
df_09

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
9,"[leave, band, follow, bust, say, simply, could...",3,0,"[Noel, Gallagher]",True,Noel Gallagher left the Manchester band follow...,5048,2009,0.0
10,"[launch, clothing, line, earlier, year, admit,...",2,0,[Liam],None,"""Liam launched his clothing line Pretty Green ...",5048,2009,0.0
11,"[thinking, next, step, musically, mind, say]",1,0,[],None,"""I'm thinking of what the next step is musical...",5048,2009,0.0
13,"[people, able, buy, record]",1,0,[],None,"""People will be able to buy his records.",5048,2009,0.0
33,"[come, back, hang, fellow, cast, member, mitch...",0,2,[Sam],None,She's coming back as Sam Mitchell and was hang...,8981,2009,1.0
...,...,...,...,...,...,...,...,...,...
19327,"[report, speculate, mime, part, track, want, p...",0,2,[],None,Some reports have speculated that she mimed pa...,2157826,2009,1.0
19328,"[know, dance, lot]",0,1,[],None,"""I know she was dancing a lot.",2157826,2009,1.0
19329,"[know, mime, think, really, great, performance...",0,2,[],None,I don't know if she was miming or not but I th...,2157826,2009,1.0
19330,"[legend, make, comeback, show, rendition, new,...",2,0,"[Pop, Robbie]",True,Pop legend Robbie Williams made his comeback o...,2157826,2009,0.0


In [6]:
def subjectivity(sentence):
    subjectivity = ""

    subjectivity = TextBlob(sentence).sentiment.subjectivity

    return subjectivity

def polarity(sentence):
    polarity = ""
    polarity = TextBlob(sentence).sentiment.polarity
    return polarity

In [28]:
#df_09['subjectivity'] = df_09['sentences'].apply(subjectivity)
df_09['polarity'] = df_09['sentences'].apply(polarity)

In [29]:
df_09["polarity"].describe()

count    5342.000000
mean        0.094567
std         0.265428
min        -1.000000
25%         0.000000
50%         0.000000
75%         0.224497
max         1.000000
Name: polarity, dtype: float64

In [30]:
#create a 2 point criteria from 0 to 1 (range of subjectivity)
def map_sentiment(value):
    if value >= 0.5: 
        return 1
    elif value <= -0.33:
        return -1
    else:
        return 0


df_09['sentiment'] = df_09['polarity'].apply(map_sentiment)
df_09['sentiment'] = df_09["sentiment"].astype(float)
df_09["sentiment"].value_counts()

 0.0    4562
 1.0     549
-1.0     231
Name: sentiment, dtype: int64

**RNN LSTM Model for Sentiment Analysis** 

This classifier identifies the polarity of the sentences based on the subjectivity. It will be used to cross-reference sentiment along with the male or female nature of the columns. 

In [ ]:
#BEST LSTM FUNCTION I COULD FINETUNE
def rnn_lstm(df, sentences_col, sentiment_col, gender_col):
    # Start timer 
    start_time = time.time()

    X = df[sentences_col]
    y = df[sentiment_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #gender col 
    gender_col = df[gender_col]
    
    # Tokenize the data
    tokenizer = Tokenizer(num_words=1000)
    tokenizer.fit_on_texts(X_train)
    
    # Convert the texts to sequences
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    
    # Pad the sequences to ensure equal length
    maxlen = 100
    X_train_padded = pad_sequences(X_train_seq, padding='post', maxlen=maxlen)
    X_test_padded = pad_sequences(X_test_seq, padding='post', maxlen=maxlen)
    
    # Define the LSTM model
    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim=1000, output_dim=64, input_length=maxlen))
    model_lstm.add(LSTM(64))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy', "mean_squared_error"])
    
    # Train the model
    model_lstm.fit(X_train_padded, y_train, epochs=3, batch_size=32, validation_data=(X_test_padded, y_test))
    
    # Evaluate the model
    score = model_lstm.evaluate(X_test_padded, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Add early stopping 
    earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

    # Make predictions on the padded sequences
    y_pred = model_lstm.predict(X_test_padded)
    
    # Convert sequences back to strings for TextBlob analysis
    X_train_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_train_seq]
    X_test_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_test_seq]
    
    # Calculate the sentiment polarity of each sentence using TextBlob
    X_train_polarity = [TextBlob(sent).sentiment.polarity for sent in X_train_str]
    X_test_polarity = [TextBlob(sent).sentiment.polarity for sent in X_test_str]
    
    # Convert polarity values to sentiment labels
    y_train_sentiment = ['positive' if polarity > 0.33 else 'negative' if polarity <- 0.33 else 'neutral' for polarity in X_train_polarity]
    y_test_sentiment = ['positive' if polarity > 0.33 else'negative' if polarity <- 0.33 else 'neutral' for polarity in X_test_polarity]
                        

    # Create a DataFrame of the test data, predicted sentiment, and gender
    df_results = pd.DataFrame({'Sentence': X_test, 'Sentiment': y_test_sentiment, 'Gender': gender_col[X_test.index]}) #,'Predicted Sentiment': y_test_sentiment.flatten()})
    
    # End timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_results


In [ ]:
senti_df09 = rnn_lstm(df_09, "sentences", "sentiment", "col_type")
senti_df09

**CNN Model for Sentiment Analysis (WE SHOULD USE THIS)**

This CNN loads a dataset of sentences within documents labeled as positive or negative, tokenizes the text, and uses a CNN to classify the sentiment of the text. It trains the model, evaluates it on a test set, and makes predictions. 

In [31]:
#CNN using rule-bases (Textblob approach)
def cnn(df):
    # Start timer 
    start_time = time.time()
    
    #virgin code CNN
    X = df['sentences']
    y = df['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the data
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)

# Convert the texts to sequences
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure equal length
    maxlen = 100
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Define the CNN model
    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "mean_squared_error"])

# Train the model
    model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

    y_pred = model.predict(X_test)

# Evaluate the model
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    #print(type(X_test))

    X_test = tokenizer.sequences_to_texts(X_test)


# Get the corresponding gender value
    X_t = pd.DataFrame(X_test)
    gender = df.iloc[X_t.index]['col_type']
    #print(type(gender))
    y_p = pd.DataFrame(y_pred, columns = ["Senti"])
    g = pd.DataFrame(gender)

    #save model 
    #with open('results_sentiment.pkl', 'wb') as f:
        #pickle.dump({'accuracy': accuracy, 'report': class_report}, f)

    # End timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return X_t, y_p, g


In [32]:
cnn_result = cnn(df_09)

Epoch 1/3
67/67 [==============================] - 1s 18ms/step - loss: 0.2869 - accuracy: 0.8516 - mean_squared_error: 0.1631 - val_loss: 0.2117 - val_accuracy: 0.8438 - val_mean_squared_error: 0.1506
Epoch 2/3
67/67 [==============================] - 1s 16ms/step - loss: 0.1524 - accuracy: 0.8570 - mean_squared_error: 0.1285 - val_loss: 0.0797 - val_accuracy: 0.8494 - val_mean_squared_error: 0.1300
Epoch 3/3
34/34 [==============================] - 0s 3ms/step
Test loss: -0.29654958844184875
Test accuracy: 0.8877455592155457

Execution time: 4.21 seconds


In [49]:
#convert the returned data into DFs
df0 = cnn_result[0]
df1 = cnn_result[1]
df2 = cnn_result[2]

In [50]:
#join the 3 DFs
joined_df = df0.join(df1, how='outer').join(df2, how='outer')
joined_df = joined_df.dropna(how='any')
joined_df

,0,Senti,col_type
9,she thinks alesha dixon will be good for the c...,0.982105,0.0
10,i've lost count to be honest he said,0.999341,0.0
11,he'd more money in and says he can get it back...,0.035793,0.0
13,police say that are often prison while the wom...,0.039656,0.0
33,it seems like all there is is customer service...,0.201409,1.0
...,...,...,...
1054,this nothing to you' she said,0.026914,0.0
1056,dressed in a face and boiler suit this he's co...,0.037852,0.0
1060,my passion is so i've been looking for a job i...,0.004971,0.0
1061,although the comedian isn't planning on a retu...,0.011709,0.0


**Read Me** we have to manually re-create the DF otherwise the columns would be merged with NaN values in different columns because of indexing. Cleaner this way. Also, There are a lot of neutral and mixed sentences (not of col_type exclusively F or M) so when we include the gender element we lose A LOT OF DATA. We should discuss this, as the gender column is important for the analysis. 

**Table Comparing Model Results**

In [52]:
results = {' ': ['LSTM', 'CNN'],
           'Accuracy': [0.638, 0.926],
           'Mean Squared Error': [0.193, 0.0605],
           'Test loss': [0.65, 0.223]}

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(results)

# Set the index of the dataframe to the Kernel column
df.set_index(' ', inplace=True)

# Display the dataframe
print(df)

      Accuracy  Mean Squared Error  Test loss
                                             
LSTM     0.638              0.1930      0.650
CNN      0.926              0.0605      0.223


In [27]:
# Import the required libraries - method with NLP Cloud 
!pip install google-cloud-language
from google.cloud import language_v1

# Instantiate the NLP client
client = language_v1.LanguageServiceClient()

# Load the text dataset
data = df_09

# Define a function to perform sentiment analysis using the Google Cloud NLP API
def analyze_sentiment(text):
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment.score
    return sentiment

# Apply the sentiment analysis function to the text data
data['sentiment'] = data['text'].apply(analyze_sentiment)

# Define a threshold for labeling sentiment as positive or negative
threshold = 0.2

# Label the sentiment as positive or negative based on the threshold
data.loc[data['sentiment'] >= threshold, 'sentiment_label'] = 'positive'
data.loc[data['sentiment'] < threshold, 'sentiment_label'] = 'negative'

# Export the labeled data to a new CSV file
data.to_csv('labeled_text_data.csv', index=False)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 883.3 kB/s eta 0:00:00a 0:00:01


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started